In [ ]:
% matplotlib inline

In [ ]:
import sys
sys.path

In [11]:
import math
import sys
import torch
from torch import nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.nn import Linear, LayerNorm, ReLU
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from torch_geometric.data import Data

import torch_sparse
from torch_sparse import SparseTensor, matmul, fill_diag, sum as sparsesum, mul
from torch_scatter import scatter
from torch_scatter import scatter_add

from torch_geometric.nn import MessagePassing, GCNConv
from torch_geometric.utils import add_self_loops, degree, softmax
from torch_geometric.nn import global_add_pool, global_mean_pool, global_max_pool
from torch_geometric.utils.num_nodes import maybe_num_nodes

In [ ]:
num_atom_type = 180 # including the extra mask tokens
num_chirality_tag = 10

num_bond_type = 5 # including aromatic and self-loop edge
num_bond_direction = 3 


def gcn_norm(edge_index, num_nodes=None):
    num_nodes = maybe_num_nodes(edge_index, num_nodes)

    edge_weight = torch.ones((edge_index.size(1), ), device=edge_index.device)

    row, col = edge_index[0], edge_index[1]
    deg = scatter_add(edge_weight, col, dim=0, dim_size=num_nodes)
    deg_inv_sqrt = deg.pow_(-0.5)
    deg_inv_sqrt.masked_fill_(deg_inv_sqrt == float('inf'), 0)
    return edge_index, deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]


class GCNConv(MessagePassing):
    def __init__(self, emb_dim, aggr="add"):
        super(GCNConv, self).__init__()
        self.emb_dim = emb_dim
        self.aggr = aggr

        self.weight = Parameter(torch.Tensor(emb_dim, emb_dim))
        self.bias = Parameter(torch.Tensor(emb_dim))
        self.reset_parameters()

        self.edge_embedding1 = nn.Embedding(num_bond_type, 1)
        self.edge_embedding2 = nn.Embedding(num_bond_direction, 1)

        nn.init.xavier_uniform_(self.edge_embedding1.weight.data)
        nn.init.xavier_uniform_(self.edge_embedding2.weight.data)

    def reset_parameters(self):
        # glorot(self.weight)
        # zeros(self.bias)
        stdv = math.sqrt(6.0 / (self.weight.size(-2) + self.weight.size(-1)))
        self.weight.data.uniform_(-stdv, stdv)
        self.bias.data.fill_(0)

    def forward(self, x, edge_index, edge_attr):
        # add self loops in the edge space
        edge_index = add_self_loops(edge_index, num_nodes=x.size(0))[0]
        # print(edge_index)

        # add features corresponding to self-loop edges.
        self_loop_attr = torch.zeros((x.size(0), edge_attr.size(1)), dtype=torch.long)
        self_loop_attr[:,0] = 4 #bond type for self-loop edge
        self_loop_attr = self_loop_attr.to(edge_attr.device).to(edge_attr.dtype)
        edge_attr = torch.cat((edge_attr, self_loop_attr), dim=0)
        
        # print(edge_attr)
        edge_embeddings = self.edge_embedding1(edge_attr[:,0]) + self.edge_embedding2(edge_attr[:,1])

        edge_index, __ = gcn_norm(edge_index)
        # print(edge_index)

        x = x @ self.weight

        # propagate_type: (x: Tensor, edge_attr: OptTensor)
        out = self.propagate(edge_index, x=x, edge_attr=edge_embeddings, size=None)

        if self.bias is not None:
            out += self.bias

        return out

    def message(self, x_j, edge_attr):
        # return x_j if edge_attr is None else edge_attr.view(-1, 1) * x_j
        return x_j if edge_attr is None else edge_attr + x_j

    def message_and_aggregate(self, adj_t, x):
        return torch_sparse.matmul(adj_t, x, reduce=self.aggr)

class GCN(nn.Module):
    def __init__(self, num_layer=5, emb_dim=64, feat_dim=64, drop_ratio=0, pool='mean', num_atom_features=9):
        super(GCN, self).__init__()
        self.num_layer = num_layer
        self.emb_dim = emb_dim
        self.feat_dim = feat_dim
        self.drop_ratio = drop_ratio
        self.num_atom_features = num_atom_features

        # Embedding layers for categorical features
        self.x_embedding1 = nn.Embedding(num_atom_type, emb_dim)
        self.x_embedding2 = nn.Embedding(num_chirality_tag, emb_dim)

        # Linear transformation for continuous features
        self.atom_feature_lin = nn.Linear(num_atom_features-2, emb_dim)

        # GCN layers
        self.gnns = nn.ModuleList()
        for layer in range(num_layer):
            self.gnns.append(GCNConv(emb_dim, aggr="add"))

        # Batch normalization layers
        self.batch_norms = nn.ModuleList()
        for layer in range(num_layer):
            self.batch_norms.append(nn.BatchNorm1d(emb_dim))

        # Pooling method
        self.pool = global_mean_pool if pool == 'mean' else global_add_pool if pool == 'add' else global_max_pool

        # Output layers
        self.feat_lin = nn.Linear(self.emb_dim, self.feat_dim)
        self.out_lin = nn.Sequential(
            nn.Linear(self.feat_dim, self.feat_dim), 
            nn.ReLU(inplace=True), 
            nn.Linear(self.feat_dim, self.feat_dim // 2)
        )

    def forward(self, data):
        x = data.x
        edge_index = data.edge_index
        edge_attr = data.edge_attr.to(torch.long)
        # print(data.edge_index)

        
        h_type = self.x_embedding1(x[:, 0].to(torch.long))
        # print(h_type.shape)
        h_chirality = self.x_embedding2(x[:, 1].to(torch.long))
        # print(h_chirality.shape)
        h_cont = self.atom_feature_lin(x[:, 2:].to(torch.float))  # 假设前两列是分类特征
        
        # print(h_cont.shape)
        
        h = h_type + h_chirality + h_cont
        # print(edge_index)
        # GCN layers
        for layer in range(self.num_layer):
            h = self.gnns[layer](h, edge_index, edge_attr)
            h = self.batch_norms[layer](h)
            h = F.dropout(F.relu(h), self.drop_ratio, training=self.training)

        # Pooling
        h = self.pool(h, data.batch)
        # print(h.shape)
        # Apply final linear transformations
        h = self.feat_lin(h)
        out = self.out_lin(h)

        return h, out

In [ ]:
def atom_features(atom):
    """Return the feature vector for a single atom"""
    atomic_num = atom.GetAtomicNum()
    
    return [
        atomic_num,  
        atom.GetDegree(),  
        atom.GetExplicitValence(), 
        atom.GetFormalCharge(), 
        atom.GetNumRadicalElectrons(), 
        # electronegativity.get(atomic_num, 0),  
        atom.GetHybridization().real,  
        1 if atom.GetChiralTag() != Chem.ChiralType.CHI_UNSPECIFIED else 0,  # Chirality
        1 if atom.IsInRing() else 0,  
        1 if atom.GetIsAromatic() else 0,  

        # ... 
    ]


In [ ]:
def bond_features(bond):
    """Return the feature vector for a single bond"""
    bt = bond.GetBondType()
    return [
        1 if bt == Chem.rdchem.BondType.SINGLE else 0,
        1 if bt == Chem.rdchem.BondType.DOUBLE else 0,
        1 if bt == Chem.rdchem.BondType.TRIPLE else 0,
        1 if bt == Chem.rdchem.BondType.AROMATIC else 0,
        # ... 
    ]

In [ ]:
def smiles_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    # Node features
    
    x = torch.tensor([atom_features(atom) for atom in mol.GetAtoms()], dtype=torch.float)

    edge_indices = []
    edge_attrs = []
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        edge_indices.append((i, j))
        edge_indices.append((j, i))
        edge_attrs += [bond_features(bond), bond_features(bond)]

    edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()
    edge_attr = torch.tensor(edge_attrs, dtype=torch.float)

    # edge_index=edge_index, 
    # Graph data object
    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
    return data


In [ ]:
from rdkit import Chem
from rdkit.Chem import rdchem
import numpy as np
electronegativity = {
    1: 2.20,  # H
    6: 2.55,  # C
    7: 3.04,  # N
    8: 3.44,  # O
    9: 3.98,  # F
    15: 2.19, # P
    16: 2.58, # S
    17: 3.16, # Cl
    35: 2.96, # Br
    53: 2.66, # I
    34: 2.55, # Se
    # ... other elements ...
}

def extract_heteroatom_electronegativity(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return []

    heteroatoms_electronegativity = []
    for atom in mol.GetAtoms():
        atomic_num = atom.GetAtomicNum()
        
        if atomic_num not in [1, 6]:
            electroneg = electronegativity.get(atomic_num, 0)
            heteroatoms_electronegativity.append(electroneg)

    return heteroatoms_electronegativity

In [ ]:

df = pd.read_csv('data/NFAs_paper.csv')


model = GCN(num_layer=3, emb_dim=64, feat_dim=64)

features = []
for idx, row in df.iterrows():
    smiles = row['SMILES']
    data = smiles_to_graph(smiles)
    # print(data.x.shape)
    electronegativities = extract_heteroatom_electronegativity(smiles)

    if data is not None:
        feature, _ = model(data)
        feature_np = feature.detach().numpy()

        
        if len(electronegativities) > 0:
            electronegativities_np = np.array([electronegativities])
            feature_np = feature_np.reshape(1, -1)  
            combined_feature = np.concatenate([feature_np, electronegativities_np], axis=1)
        else:
            combined_feature = feature_np.reshape(1, -1)  

        features.append(combined_feature)
    else:
        features.append(None)  


df['Features'] = features


df.to_csv('data/GCN_NFAs_paper.csv', index=False)